In [4]:
# reading the rtf file
with open('algoparams_from_ui.json.rtf','r') as file:
    rtf_d = file.read()
print(rtf_d)

{\rtf1\adeflang1025\ansi\ansicpg1252\uc1\adeff31507\deff0\stshfdbch31506\stshfloch31506\stshfhich31506\stshfbi31507\deflang1033\deflangfe1033\themelang1033\themelangfe0\themelangcs0{\fonttbl{\f2\fbidi \fmodern\fcharset0\fprq1{\*\panose 02070309020205020404}Courier New;}{\f34\fbidi \froman\fcharset0\fprq2{\*\panose 02040503050406030204}Cambria Math;}
{\f37\fbidi \fswiss\fcharset0\fprq2{\*\panose 020f0502020204030204}Calibri;}{\f43\fbidi \fmodern\fcharset0\fprq1{\*\panose 020b0609020204030204}Consolas;}{\flomajor\f31500\fbidi \froman\fcharset0\fprq2{\*\panose 02020603050405020304}Times New Roman;}
{\fdbmajor\f31501\fbidi \froman\fcharset0\fprq2{\*\panose 02020603050405020304}Times New Roman;}{\fhimajor\f31502\fbidi \fswiss\fcharset0\fprq2{\*\panose 020f0302020204030204}Calibri Light;}
{\fbimajor\f31503\fbidi \froman\fcharset0\fprq2{\*\panose 02020603050405020304}Times New Roman;}{\flominor\f31504\fbidi \froman\fcharset0\fprq2{\*\panose 02020603050405020304}Times New Roman;}
{\fdbminor\f3

In [3]:
# extracting json content and creating new json file
# for extracting json content converting the rtf file into text format and applying necessary removal for the content
from striprtf.striprtf import rtf_to_text
import json
import re


In [6]:
text = rtf_to_text(rtf_d)
print(text)

{
    "session_name": "test",
    "session_description": "test",
    "design_state_data": {

      "session_info" : {
        "project_id": "1",
        "experiment_id": "kkkk-11",
        "dataset":"iris_modified.csv",
        "session_name": "test",
        "session_description": "test"
        },

      "target": {
        "prediction_type": "Regression",
        "target": "petal_width",
        "type":"regression",
        "partitioning": true
      },
      "train": {
        "policy": "Split the dataset",
        "time_variable": "sepal_length",
        "sampling_method": "No sampling(whole data)",
        "split": "Randomly",
        "k_fold": false,
        "train_ratio": 0,
        "random_seed": 0
      },
      "metrics": {
        "optomize_model_hyperparameters_for": "AUC",
        "optimize_threshold_for": "F1 Score",
        "compute_lift_at": 0,
        "cost_matrix_gain_for_true_prediction_true_result": 1,
        "cost_matrix_gain_for_true_prediction_false_result": 0,

In [11]:
# so json content is in the {}

jsonformat = r'{.*}'  
match = re.search(jsonformat, text, re.DOTALL)
print(match)

<built-in method start of re.Match object at 0x00000204CF0ED0C0>


In [12]:
# now we check the data extracted data is valid jason or not
import json

In [28]:
import json
def check_data(data):
    try:
        json.loads(json_data)
        return True  
    except json.JSONDecodeError as e:
        print(f"Invalid JSON: {e}")
        return False  

In [29]:
data  = text

In [30]:
if validate_json_syntax(data):
    print("valid data ")
else:
    print("invalid data")

valid data 


In [39]:
# saving this data into json file for futher calculation

In [40]:
def save_json_to_file(data,path):
    with open(output_file_path, 'w') as json_file:
        json.dump(text, json_file, indent=4)
    print(f"JSON data successfully saved to {output_file_path}")
    
path = 'algoparams_from_ui.json.rtf'
extract_file_path = 'data.json'

In [41]:
save_json_to_file(text,output_file_path)

JSON data successfully saved to data.json


### 1) Read the target and type of regression to be run

In [63]:
with open('data.json','r') as file:
    json_data = json.load(file)


In [64]:
parsed_json = json.loads(json_data)
target = parsed_json.get("design_state_data", {}).get("target", {})
prediction_type = target.get("prediction_type")
regression_type = target.get("type")
target_variable = target.get("target")


In [65]:
print(target)

{'prediction_type': 'Regression', 'target': 'petal_width', 'type': 'regression', 'partitioning': True}


In [66]:
print(prediction_type)

Regression


In [67]:
print(regression_type)

regression


In [68]:
print(target_variable)

petal_width


## 2) Read the features (which are column names in the csv) and figure out what missing imputation needs to be applied and apply that to the columns loaded in a dataframe

In [72]:
with open('data.json','r') as file:
    json_data = json.load(file)


In [75]:
parsed_json = json.loads(json_data)

In [82]:
feature_handling = parsed_json.get('design_state_data', {}).get('feature_handling', {})

In [83]:
for feature_name, feature_details in feature_handling.items():
    if feature_details.get('is_selected', False):
        feature_detail_data = feature_details.get('feature_details', {})
        imputation_strategy = feature_detail_data.get('impute_with')
        if imputation_strategy == "Average of values":
            df[feature_name] = df[feature_name].fillna(df[feature_name].mean())
        elif imputation_strategy == "custom":
            impute_value = feature_detail_data.get('impute_value', 0)
            df[feature_name] = df[feature_name].fillna(impute_value)

In [84]:
print(df.head())

   sepal_length  sepal_width  petal_length  petal_width      species
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa


## 3) Compute feature reduction based on input. See the screenshot below where there can be No Reduction, Corr with Target, Tree-based, PCA. Please make sure you write code so that all options can work. If we rerun your code with a different Json it should work if we switch No Reduction to say PCA.

In [85]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [95]:
le = LabelEncoder()
df['species_encoded'] = le.fit_transform(df['species'])
 


In [96]:
df.head()

sepal_length  sepal_width  petal_length  petal_width  species  \
0           5.1          3.5           1.4          0.2        0   
1           4.9          3.0           1.4          0.2        0   
2           4.7          3.2           1.3          0.2        0   
3           4.6          3.1           1.5          0.2        0   
4           5.0          3.6           1.4          0.2        0   

   species_encoded  
0                0  
1                0  
2                0  
3                0  
4                0

In [102]:
df.head()

sepal_length  sepal_width  petal_length  petal_width  species_encoded
0           5.1          3.5           1.4          0.2                0
1           4.9          3.0           1.4          0.2                0
2           4.7          3.2           1.3          0.2                0
3           4.6          3.1           1.5          0.2                0
4           5.0          3.6           1.4          0.2                0

In [128]:
def apply_feature_reduction(df, target, red_method, redu_params):
    if red_method == "No Reduction":
        print("No feature reduction applied.")
        return df
    
    elif red_method == "Corr with Target":
        corr_threshold = redu_params.get("corr_threshold", 0.1)
        target_corr = df.corrwith(target)
        selected_features = target_corr[target_corr.abs() > corr_threshold].index
        return df[selected_features]
    elif red_method == "Tree-based":
        num_features_to_keep = int(redu.get("num_of_features_to_keep", 4))
        rf_model = RandomForestRegressor(n_estimators=10, random_state=0)
        rf_model.fit(df, target)
        important_features = df.columns[np.argsort(rf_model.feature_importances_)[-num_features_to_keep:]]
        return df[important_features]
              
    elif redu_method == "PCA":
        pca = PCA(n_components=redu_params.get("num_components", 2))
        reduced_data = pca.fit_transform(df)
    return pd.DataFrame(reduced_data, columns=[f"PCA_{i+1}" for i in range(reduced_data.shape[1])])

        

In [129]:
# get the feature reduction method and parameters
reduction_config = parsed_json.get("design_state_data", {}).get("feature_reduction", {})
reduction_method = reduction_config.get("feature_reduction_method", "No Reduction")
reduction_params = {
        "num_of_features_to_keep": reduction_config.get("num_of_features_to_keep", 4),
        "corr_threshold": reduction_config.get("corr_threshold", 0.1),
        "num_components": reduction_config.get("num_components", 2)
    }

    

In [130]:
target_column = parsed_json.get("design_state_data", {}).get("target", {}).get("target", None)

if target_column not in df.columns:
    target = df[target_column]
    features = df.drop(columns=[target_column])
    features = encode_categorical_columns(features)
    reduced_features = apply_feature_reduction(features, target, red_method, redu_params)
    print(reduced_features)    

## 4) Parse the Json and make the model objects (using sklean) that can handle what is required in the “prediction_type” specified in the JSON (See #1 where “prediction_type” is specified). Keep in mind not to pick models that don’t apply for the prediction_type specified

In [134]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR


def chose_model(prediction_type):
   
    if prediction_type == "classification":
        print("Creating a model for classification.")
        model = RandomForestClassifier(n_estimators=100, random_state=0)  
    elif prediction_type == "regression":
        print("Creating a model for regression.")
        model = RandomForestRegressor(n_estimators=100, random_state=0) 
    else:
        print("model Not found")

In [135]:
prediction_type = prediction_type.lower()
model = chose_model(prediction_type)
print("Generated Model Object:")
print(model)        



Creating a model for regression.
Generated Model Object:
None


## 5) Run the fit and predict on each model – keep in mind that you need to do hyper parameter tuning i.e., use GridSearchCV##

In [136]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [137]:
def get_model_and_params(prediction_type):
    if prediction_type == "classification":
        model = RandomForestClassifier(random_state=0)
        param_grid = {
            "n_estimators": [50, 100, 150],
            "max_depth": [None, 10, 20],
            "min_samples_split": [2, 5, 10]
        }
    elif prediction_type == "regression":
        model = RandomForestRegressor(random_state=0)
        param_grid = {
            "n_estimators": [50, 100, 150],
            "max_depth": [None, 10, 20],
            "min_samples_split": [2, 5, 10]
        }
    else:
        print('None')
    
    return model, param_grid

In [138]:
target = df[target_column]
features = df.drop(columns=[target_column])

In [140]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

In [142]:
from sklearn.preprocessing import StandardScaler
if prediction_type == "regression":
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

In [143]:
model, param_grid = get_model_and_params(prediction_type)

In [146]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring="accuracy" if prediction_type == "classification" else "neg_mean_squared_error")
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=0),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]},
             scoring='neg_mean_squared_error')

In [147]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

In [154]:
if prediction_type == "classification":
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Accuracy: {accuracy:}")
    print(f"Precision: {precision:}")
    print(f"Recall: {recall:}")
    print(f"F1 Score: {f1:}")

elif prediction_type == "regression":
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"MSE: {mse:}")
    print(f"MAE: {mae:}")
    print(f"R2: {r2:}")


MSE: 0.035460496132643654
MAE: 0.13150447132388307
R2: 0.9261393540249039


In [151]:
 print("\nBest Hyperparameters:", grid_search.best_params_)


Best Hyperparameters: {'max_depth': None, 'min_samples_split': 10, 'n_estimators': 50}
